In [17]:
import os
import sys

import numpy as np

import pandas as pd
import glob

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model, model_from_json
from keras.layers import Dense, GlobalAveragePooling2D
from keras import optimizers

basemodel_layer_num = 311 #corresponding to len(base_model.layers)

Using TensorFlow backend.


In [10]:
BASE_MODEL_PATH="trained_model"
%mkdir -p $BASE_MODEL_PATH

In [5]:
from models.modelutils import ModelCompiler

In [6]:
compiler = ModelCompiler(BASE_MODEL_PATH)

In [7]:
cat = "bay"

In [11]:
model_path = "{}/model_{}".format(BASE_MODEL_PATH, cat)

In [8]:
model = compiler.generate_compiled_model(model_path)

87916544/87910968 [==============================] - 12s 0us/step


In [13]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()

In [15]:
# modelutils' is different extension.

def dir2filedict(basedir):
    res = {}
    for f in glob.iglob("{}/**/*".format(basedir), recursive=True):
        cat = os.path.basename(os.path.dirname(f))
        res.setdefault(cat, []).append(f)
    return res

In [18]:
fdict = dir2filedict("data")

In [23]:
catecories = sorted(fdict.keys())

In [24]:
catecories

['bay',
 'beach',
 'birds',
 'boeing',
 'buildings',
 'city',
 'clouds',
 'data',
 'f-16',
 'face',
 'helicopter',
 'mountain',
 'ocean',
 'ships',
 'sky',
 'sunrise',
 'sunset']

In [39]:
TRAIN_VALID_RATIO=0.9

In [25]:
import random

In [50]:
def split_files(input_paths, ratio= TRAIN_VALID_RATIO):
    paths = sorted(input_paths)
    random.shuffle(paths)
    sep = int(len(paths)*ratio)
    return paths[0:sep], paths[sep:]

In [55]:
def split_fdict(fdict):
    trdict = {}
    valdict = {}
    cats = sorted(fdict.keys())
    for cat in cats:
        tr, val = split_files(fdict[cat])
        trdict[cat] = tr
        valdict[cat] = val
    return trdict, valdict

In [60]:
random.seed(123)
trdict, valdict = split_fdict(fdict)